## Получаем данные

In [1]:
import pandas as pd

In [2]:
import numpy as np
from tqdm import tqdm_notebook

In [3]:
df_ratings = pd.read_csv('ratings1M.csv', sep='::', names=['userId', 'movieId', 'rating', 'timestamp'])
df_movies = pd.read_csv('movies1M.csv', sep='::', names=['movieId', 'title', 'genres'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [4]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
df = pd.merge(df_ratings, df_movies, on='movieId')

In [6]:
del df['timestamp']
del df['genres']

In [7]:
df.head()

,userId,movieId,rating,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975)
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975)
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975)
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975)


## User 2 Item

In [17]:
from surprise import Dataset, Reader, KNNBasic, KNNWithMeans, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import KFold

In [9]:
df_ratings.describe()

,userId,movieId,rating,timestamp
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.865540e+03,3.581564e+00,9.722437e+08
std,1.728413e+03,1.096041e+03,1.117102e+00,1.215256e+07
min,1.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08
25%,1.506000e+03,1.030000e+03,3.000000e+00,9.653026e+08
50%,3.070000e+03,1.835000e+03,4.000000e+00,9.730180e+08
75%,4.476000e+03,2.770000e+03,4.000000e+00,9.752209e+08
max,6.040000e+03,3.952000e+03,5.000000e+00,1.046455e+09


In [10]:
df_for_surpise = df_ratings[['userId', 'movieId', 'rating']]

In [11]:
df_for_surpise.columns = ['uid', 'iid', 'rating']

In [12]:
df_for_surpise.head()

,uid,iid,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [13]:
reader = Reader(rating_scale=(1, 5))

In [14]:
dataset = Dataset.load_from_df(df_for_surpise, reader)

In [18]:
scores = []
kfold = KFold(5)
for trainset, testset in tqdm_notebook(kfold.split(dataset)):
    #algo = KNNWithMeans(k=20, sim_options={'name': 'SVD', 'user_based': True})
    algo = SVD()
    algo.fit(trainset)
    predictions = algo.test(testset)
    scores.append(accuracy.rmse(predictions))
    
sum(scores)/5

RMSE: 0.8747
RMSE: 0.8734
RMSE: 0.8756
RMSE: 0.8736
RMSE: 0.8720



0.8738699924161274

In [19]:
scores

[0.874723614855214,
 0.8733952103061573,
 0.8756164362065016,
 0.8735904157836326,
 0.872024284929132]